In [2]:
import numpy as np
import os
import scipy.stats as stats
import scipy.io as io
import networkx as nx
import random
import itertools
import glob
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
from scipy.io import loadmat, savemat
from collections import Counter
from collections import defaultdict
from numpy import arange,array,ones,linalg
from sklearn.linear_model import LinearRegression
import pandas as pd
import powerlaw
import os.path
from os import path
from scipy.optimize import curve_fit
import numpy as np

In [3]:
##Calculate entropy
def Entropy_MC(s,b,k,beta,gamma):
    inv, l, n = k-b, len(s), 4**k
    T=np.zeros((n,n))
    count = [0]*n
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd = dict(zip(word_list, list(range(len(word_list)))))
    uu = []
    for i in range(k,l-b):
        n1, n2 = wd[s[i-k:i]], wd[s[i-k+b:i+b]]
        T[n1,n2] += 1
        count[n1] += 1
        
    wo_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    
    for i in range(n):
        wo = word_list[i]
        for l in wo_next:
            j = wd[wo[b:]+l]
            T[i,j] = (T[i,j]+beta)/(count[i]+4**b*beta)
    sum_count = np.sum(count)+4**k*gamma
    
    prob = [(count[i]+gamma)/sum_count for i in range(len(count))] 
    return T,prob

In [4]:
from collections import defaultdict
def getSeqs(filename):
    seq_list = defaultdict(list)
    mapseq_list =  defaultdict(list)
    mapping = {'A': 'a', 'T': 't', 'C': 'c', 'G': 'g','a': 'a', 't': 't', 'c': 'c', 'g': 'g'}
    with open(filename) as f:
        j = -1
        for i, line in enumerate(f):
            if line.startswith('>'):
                j += 1
            else:
                this_line = list(line)
                this_line = list(filter(lambda ch: ch in 'acgtACGT', this_line))
                seq_list[j].extend(this_line)
                mapseq_list[j] = ''.join(list(map(lambda ch: mapping[ch], seq_list[j])))
    return mapseq_list

In [5]:
def stats(s,b,k,beta,gamma):
    T_list = []
    prob_list=[]
    for i in range(len(s)):
        T,prob = Entropy_MC(s[i],b,k,beta,gamma)
        T_list.append(T)
        prob_list.append(prob)
    return T_list,prob_list

In [13]:
def seq_generate(folder):
    filenames= glob.glob(folder+'/2020*.fasta')
    filenames.sort()
    print(filenames)
    seq = []
    for filename in filenames:
        s = getSeqs(filename)
        print(len(s))
        seq.append(s)
    return seq

In [ ]:
seq = seq_generate('USA') # change to seq = seq_generate('Europe') for European Samples

In [15]:
def high_transitions(seq,b,k,beta,gamma,folder):
    n = 4**k
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd = dict(zip(word_list, list(range(len(word_list)))))
    wo_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    ctr = 0
    sseq = []
    m_val = 0
    for s in seq:
        if len(s)>=10:
            sseq.append(s)
            if len(s) > m_val:
                m_val = len(s)
    
    d = len(sseq)
    
    a_mean = np.zeros((d,n))
    c_mean = np.zeros((d,n))
    g_mean = np.zeros((d,n))
    t_mean = np.zeros((d,n))
    a_std = np.zeros((d,n))
    c_std = np.zeros((d,n))
    g_std = np.zeros((d,n))
    t_std = np.zeros((d,n))
    f_a_mean = np.zeros((d,n))
    f_c_mean = np.zeros((d,n))
    f_g_mean = np.zeros((d,n))
    f_t_mean = np.zeros((d,n))
    f_a_std = np.zeros((d,n))
    f_c_std = np.zeros((d,n))
    f_g_std = np.zeros((d,n))
    f_t_std = np.zeros((d,n))
    pp_mean = []
    pp_std = []
    ent_mean = []
    ent_std = []
    for s in sseq:
        print('TIME_FRAME {}'.format(ctr))
        T_list,prob_list = stats(s,b,k,beta,gamma)
        pp = np.mean(prob_list,axis = 0)
        pp_mean.append(pp)
        pp = np.std(prob_list,axis = 0)
        pp_std.append(pp)
        i = 0
        AA = [[] for cc in range(n)]
        CC = [[] for cc in range(n)]
        BB = np.zeros((len(T_list),n))
        ENT = np.zeros((len(T_list),1))
        for Q in T_list:
            ct = 0
            for r in Q:
                R = r[np.nonzero(r)]
                AA[ct]+= [R]
                ct+=1
            i+=1
        
        
        
            

        UU_mean = np.zeros((n,4))
        UU_std = np.zeros((n,4))
        AA = np.array(AA)
        print(AA.shape)
        for jj in range(n):
            UU_mean[jj] = np.mean(AA[jj],axis = 0)
            UU_std[jj] = np.std(AA[jj],axis = 0)
            
        a_mean[ctr] = UU_mean[:,0]
        c_mean[ctr] = UU_mean[:,1]
        g_mean[ctr] = UU_mean[:,2]
        t_mean[ctr] = UU_mean[:,3]
        a_std[ctr] = UU_std[:,0]
        c_std[ctr] = UU_std[:,1]
        g_std[ctr] = UU_std[:,2]
        t_std[ctr] = UU_std[:,3]
        ctr+=1
        #print('------------------------------------------------------------')
    a_mean = a_mean.T
    c_mean = c_mean.T
    g_mean = g_mean.T
    t_mean = t_mean.T
    a_std = a_std.T
    c_std = c_std.T
    g_std = g_std.T
    t_std = t_std.T
    #print(ent_mean,ent_std)
    return a_mean,c_mean,g_mean,t_mean,a_std,c_std,g_std,t_std,d,n
def file_generation(a_mean,c_mean,g_mean,t_mean,a_std,c_std,g_std,t_std,d,b,k,stdev,folder):
    x = [i for i in range(d)]
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd = dict(zip(word_list, list(range(len(word_list)))))
    wo_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    n = 4**k
    ct_a = ct_c = ct_g = ct_t = 0
    tps = []
    l_hertz = []
    g_hertz = []
    for v in range(n):
        if np.std(a_mean[v]) > stdev:
            
            l_hertz.append(a_mean[v])
            g_hertz.append(a_std[v])
            ct_a+=1
            tps.append(word_list[v]+' -> a')

        if np.std(c_mean[v]) > stdev:
            l_hertz.append(c_mean[v])
            g_hertz.append(c_std[v])
            ct_c+=1
            tps.append(word_list[v]+' -> c')
            
        if np.std(g_mean[v]) > stdev:
            l_hertz.append(g_mean[v])
            g_hertz.append(g_std[v])
            ct_g+=1
            tps.append(word_list[v]+' -> g')
            
        if np.std(t_mean[v]) > stdev:
            l_hertz.append(t_mean[v])
            g_hertz.append(t_std[v])
            ct_t+=1
            tps.append(word_list[v]+' -> t')
            
            
    
    l_hertz = np.array(l_hertz)
    g_hertz = np.array(g_hertz)
    np.savetxt(folder+'/new_time_series_mean_'+str(k)+'_'+str(stdev)+'.csv', l_hertz, delimiter=",")
    np.savetxt(folder+'/new_time_series_std_'+str(k)+'_'+str(stdev)+'.csv', g_hertz, delimiter=",")
    return tps,ct_a,ct_c,ct_g,ct_t

In [ ]:
beta = 0.5
gamma = 0.5
l = [[1,4],[1,5]]
stdevs = [0.001*(1+i) for i in range(30)]

folder = 'USA' #Change to 'Europe' for European Samples
for val in l:
    a_mean,c_mean,g_mean,t_mean,a_std,c_std,g_std,t_std,d,n = high_transitions(seq,val[0],val[1],beta,gamma,folder)
    int_num = []
    for stdev in stdevs:
        aa = open(folder+'/interesting_transitions_{}_k_{}.txt'.format(stdev,val[1]),'w')
        tps,ct_a,ct_c,ct_g,ct_t = file_generation(a_mean,c_mean,g_mean,t_mean,a_std,c_std,g_std,t_std,d,val[0],val[1],stdev,folder)
        int_num.append(ct_a+ct_c+ct_g+ct_t)
        print(int_num[-1])
        aa.write('k = '+str(val[1])+'\n')
        aa.write(str(tps)+'\n')
        aa.write(str(ct_a)+' '+str(ct_c)+' '+str(ct_g)+' '+str(ct_t)+'\n')
        aa.close()
    

In [ ]:
k = 5
stdevs = [0.001*(1+i) for i in range(30)]
stdevs = np.array(stdevs)
n = 4**(k+1)
folders = ['USA','Europe']
labels = ['USA','Europe']
for i,folder in enumerate(folders):
    U =[]
    for stdev in stdevs:
        filesize = os.path.getsize(folder+'/new_time_series_mean_'+str(k)+'_'+str(stdev)+'.csv')
        if filesize == 0:
            U.append(0)
            continue
        df = pd.read_csv(folder+'/new_time_series_mean_'+str(k)+'_'+str(stdev)+'.csv')  
        rr = df.shape[0]+1
        rr = rr/n*100
        U.append(rr)
        #U = np.array(U)
    plt.rcParams.update({'font.size': 14})
    plt.loglog(stdevs,U,label = labels[i])
    plt.xlabel('Variability')
    plt.ylabel('Transitions (%age)')
plt.legend(loc = 'best')   
plt.tight_layout()
plt.savefig('Variability_US_Europe_{}_log_log.png'.format(k))
plt.show()
plt.close()

In [ ]:
def normalize(x):
        return (x - x.min(0)) / x.ptp(0)
plt.rcParams.update({'font.size': 14})
k = 5
b = 1
folders = ['USA','Europe']
labels = ['USA','Europe']
colors = ['blue','red']
xd = {}
yd2 = {}
for i,folder in enumerate(folders):
    data = loadmat(folder+'/hurst_'+str(k)+'_001.mat')
    H = data['h_list']
    g = sns.distplot(H, hist=True, kde=True, bins=int(100), color = 'green', hist_kws={'edgecolor':'black'},kde_kws={'linewidth': 4})
    line = g.get_lines()[0]
    xd[folder] = line.get_xdata()
    yd = line.get_ydata()
    yd2[folder] = normalize(yd)

    plt.xlabel('Hurst Exponent')
    plt.ylabel('Density')
    plt.title('k = {}, b = 1'.format(k))
    plt.tight_layout()
    #plt.savefig(folder+'/hurst_exponent_density_'+str(k)+'.png')
    plt.close()
for i,folder in enumerate(folders):
    plt.plot(xd[folder],yd2[folder],label = labels[i],color = colors[i]) 
plt.legend(loc = 'best')
plt.xlabel('Hurst Exponent')
plt.ylabel('Density')
plt.title('k = {}, b = 1 '.format(k))
plt.xlim(0,1)
plt.tight_layout()
plt.savefig('Hurst_US_Europe_{}.png'.format(k))
plt.show()
plt.close()